In [321]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [322]:
# Import the data
df = pd.read_csv("utah_sales_20241203_201859.csv")
df.head()

C:\Users\kerim\AppData\Local\Temp\ipykernel_5168\763977066.py:2: DtypeWarning: Columns (11,12,13,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("utah_sales_20241203_201859.csv")


,id,formattedAddress,addressLine1,addressLine2,city,state,zipCode,county,latitude,longitude,...,squareFootage,lotSize,yearBuilt,status,price,listedDate,removedDate,createdDate,lastSeenDate,daysOnMarket
0,"231-S-Virginia-St,-Kaysville,-UT-84037","231 S Virginia St, Kaysville, UT 84037",231 S Virginia St,NaN,Kaysville,UT,84037,Davis,41.031291,-111.934739,...,862.0,4792.0,1941.0,Inactive,360000,2024-01-05T00:00:00.000Z,2024-04-06T00:00:00.000Z,2023-12-08T00:00:00.000Z,2024-04-05T00:00:00.000Z,92.0
1,"3157-Monroe-Blvd,-Ogden,-UT-84403","3157 Monroe Blvd, Ogden, UT 84403",3157 Monroe Blvd,NaN,Ogden,UT,84403,Weber,41.206655,-111.959974,...,1750.0,4792.0,1941.0,Inactive,314999,2024-01-05T00:00:00.000Z,2024-04-06T00:00:00.000Z,2024-01-06T00:00:00.000Z,2024-04-05T00:00:00.000Z,92.0
2,"1988-Stone-Creek-Rd,-Unit-19,-Park-City,-UT-84098","1988 Stone Creek Rd, Unit 19, Park City, UT 84098",1988 Stone Creek Rd,Unit 19,Park City,UT,84098,Summit,40.683172,-111.549270,...,2082.0,NaN,2025.0,Inactive,2790000,2024-01-10T00:00:00.000Z,2024-04-06T00:00:00.000Z,2022-07-27T14:22:11.749Z,2024-04-05T00:00:00.000Z,87.0
3,"1990-Stone-Creek-Rd,-Unit-20,-Park-City,-UT-84098","1990 Stone Creek Rd, Unit 20, Park City, UT 84098",1990 Stone Creek Rd,Unit 20,Park City,UT,84098,Summit,40.683081,-111.549202,...,2533.0,NaN,2025.0,Inactive,3190000,2024-01-10T00:00:00.000Z,2024-04-06T00:00:00.000Z,2022-07-27T14:24:30.407Z,2024-04-05T00:00:00.000Z,87.0
4,"4398-N-2525-W,-Cedar-City,-UT-84721","4398 N 2525 W, Cedar City, UT 84721",4398 N 2525 W,NaN,Cedar City,UT,84721,Iron,37.757465,-113.097258,...,3000.0,21780.0,2021.0,Inactive,540000,2024-01-12T00:00:00.000Z,2024-04-06T00:00:00.000Z,2023-12-31T00:00:00.000Z,2024-04-05T00:00:00.000Z,85.0


In [323]:
# Drop rows with missing values
df = df.dropna()

In [324]:
# Convert all of the string data to numbers with one hot encoder or get dummies.
# df = pd.get_dummies(df, columns=['city', 'state', 'zipCode', 'county', 'latitude', 'longitude', 'propertyType',
# 'bedrooms', 'bathrooms', 'squareFootage', 'lotSize', 'yearBuilt','status', 'listedDate', 'removedDate', 'createdDate', 'daysOnMarket'])

In [325]:
df.head()


,id,formattedAddress,addressLine1,addressLine2,city,state,zipCode,county,latitude,longitude,...,squareFootage,lotSize,yearBuilt,status,price,listedDate,removedDate,createdDate,lastSeenDate,daysOnMarket
8,"13647-S-Vernet-Dr-W,---2039,-Riverton,-UT-84096","13647 S Vernet Dr W, # 2039, Riverton, UT 84096",13647 S Vernet Dr W,# 2039,Riverton,UT,84096,Salt Lake,40.503300,-112.001572,...,3622.0,1307.0,2024.0,Inactive,588900,2024-01-16T00:00:00.000Z,2024-04-06T00:00:00.000Z,2024-02-14T00:00:00.000Z,2024-04-05T00:00:00.000Z,81.0
11,"1046-E-Blazing-Sun-Dr,---324,-Eagle-Mountain,-...","1046 E Blazing Sun Dr, # 324, Eagle Mountain, ...",1046 E Blazing Sun Dr,# 324,Eagle Mountain,UT,84005,Utah,40.322655,-112.023788,...,4054.0,9148.0,2024.0,Inactive,632525,2024-01-18T00:00:00.000Z,2024-04-06T00:00:00.000Z,2024-01-19T00:00:00.000Z,2024-04-05T00:00:00.000Z,79.0
13,"3612-N-Huntsman-Path,-Unit-609,-Eden,-UT-84310","3612 N Huntsman Path, Unit 609, Eden, UT 84310",3612 N Huntsman Path,Unit 609,Eden,UT,84310,Weber,41.324364,-111.823717,...,1100.0,3485.0,2002.0,Inactive,505000,2024-01-24T00:00:00.000Z,2024-04-06T00:00:00.000Z,2021-08-19T11:30:57.821Z,2024-04-05T00:00:00.000Z,73.0
17,"4608-W-Hemsworth-Ln-S,---2067,-Riverton,-UT-84096","4608 W Hemsworth Ln S, # 2067, Riverton, UT 84096",4608 W Hemsworth Ln S,# 2067,Riverton,UT,84096,Salt Lake,40.504021,-112.000793,...,2280.0,1307.0,2024.0,Inactive,501900,2024-01-29T00:00:00.000Z,2024-04-06T00:00:00.000Z,2024-02-29T00:00:00.000Z,2024-04-05T00:00:00.000Z,68.0
24,"3155-S-Hidden-Valley-Dr,-Unit-208,-Saint-Georg...","3155 S Hidden Valley Dr, Unit 208, Saint Georg...",3155 S Hidden Valley Dr,Unit 208,Saint George,UT,84790,Washington,37.053494,-113.569428,...,1430.0,871.0,2006.0,Inactive,309900,2024-02-10T00:00:00.000Z,2024-04-06T00:00:00.000Z,2024-02-10T00:00:00.000Z,2024-04-05T00:00:00.000Z,56.0


In [326]:
# Split the data into training and testing sets
X = df.drop('price', axis=1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [327]:
#Print x_train column headers.
X_train.head()

,id,formattedAddress,addressLine1,addressLine2,city,state,zipCode,county,latitude,longitude,...,bathrooms,squareFootage,lotSize,yearBuilt,status,listedDate,removedDate,createdDate,lastSeenDate,daysOnMarket
17470,"5176-Fairview-Loop,-Unit-503,-Eden,-UT-84310","5176 Fairview Loop, Unit 503, Eden, UT 84310",5176 Fairview Loop,Unit 503,Eden,UT,84310,Weber,41.325473,-111.822823,...,3.5,2627.0,3485.0,2005.0,Inactive,2023-09-22T00:00:00.000Z,2023-11-23T00:00:00.000Z,2023-01-10T09:50:43.110Z,2023-11-22T00:00:00.000Z,62.0
66045,"1225-N-Dixie-Downs-Rd,-Unit-82,-Saint-George,-...","1225 N Dixie Downs Rd, Unit 82, Saint George, ...",1225 N Dixie Downs Rd,Unit 82,Saint George,UT,84770,Washington County,37.129390,-113.626487,...,1.0,548.0,1742.0,1999.0,Inactive,2022-08-23T05:16:24.000Z,2022-11-22T14:51:42.152Z,2022-08-23T08:17:11.002Z,2022-11-15T13:14:42.946Z,92.0
53783,"6772-W-Minos-Pl,-Unit-2,-West-Jordan,-UT-84081","6772 W Minos Pl, Unit 2, West Jordan, UT 84081",6772 W Minos Pl,Unit 2,West Jordan,UT,84081,Salt Lake,40.608983,-112.052366,...,2.5,2496.0,3049.0,2018.0,Inactive,2022-08-26T00:00:00.000Z,2023-02-28T00:00:00.000Z,2022-08-26T08:17:41.501Z,2023-02-27T00:00:00.000Z,186.0
51103,"181-E-Larsen-Ln,-Unit-4,-Harrisville,-UT-84404","181 E Larsen Ln, Unit 4, Harrisville, UT 84404",181 E Larsen Ln,Unit 4,Harrisville,UT,84404,Weber,41.272445,-111.975533,...,2.5,1986.0,10019.0,2022.0,Inactive,2023-01-03T00:00:00.000Z,2023-03-18T00:00:00.000Z,2022-08-26T09:01:42.155Z,2023-03-17T00:00:00.000Z,74.0
82005,"1497-S-Ridgeline-Dr-E,-Unit-1221,-Heber-City,-...","1497 S Ridgeline Dr E, Unit 1221, Heber City, ...",1497 S Ridgeline Dr E,Unit 1221,Heber City,UT,84032,Wasatch County,40.489217,-111.284265,...,2.0,2510.0,47916.0,2006.0,Inactive,2022-07-09T10:03:05.207Z,2022-08-13T14:59:24.283Z,2022-07-09T10:03:05.207Z,2022-08-06T07:44:54.247Z,36.0


In [328]:
# Print the first 5 rows of the y_train data.
y_train.head()

17470    809700
66045    139900
53783    489000
51103    599900
82005    889000
Name: price, dtype: object

In [329]:
# Make an X variable with all columns except price
X_full = df.drop(columns = ['price'])
X_full.columns

Index(['id', 'formattedAddress', 'addressLine1', 'addressLine2', 'city',
       'state', 'zipCode', 'county', 'latitude', 'longitude', 'propertyType',
       'bedrooms', 'bathrooms', 'squareFootage', 'lotSize', 'yearBuilt',
       'status', 'listedDate', 'removedDate', 'createdDate', 'lastSeenDate',
       'daysOnMarket'],
      dtype='object')

In [330]:
X_full.head()


,id,formattedAddress,addressLine1,addressLine2,city,state,zipCode,county,latitude,longitude,...,bathrooms,squareFootage,lotSize,yearBuilt,status,listedDate,removedDate,createdDate,lastSeenDate,daysOnMarket
8,"13647-S-Vernet-Dr-W,---2039,-Riverton,-UT-84096","13647 S Vernet Dr W, # 2039, Riverton, UT 84096",13647 S Vernet Dr W,# 2039,Riverton,UT,84096,Salt Lake,40.503300,-112.001572,...,3.0,3622.0,1307.0,2024.0,Inactive,2024-01-16T00:00:00.000Z,2024-04-06T00:00:00.000Z,2024-02-14T00:00:00.000Z,2024-04-05T00:00:00.000Z,81.0
11,"1046-E-Blazing-Sun-Dr,---324,-Eagle-Mountain,-...","1046 E Blazing Sun Dr, # 324, Eagle Mountain, ...",1046 E Blazing Sun Dr,# 324,Eagle Mountain,UT,84005,Utah,40.322655,-112.023788,...,2.5,4054.0,9148.0,2024.0,Inactive,2024-01-18T00:00:00.000Z,2024-04-06T00:00:00.000Z,2024-01-19T00:00:00.000Z,2024-04-05T00:00:00.000Z,79.0
13,"3612-N-Huntsman-Path,-Unit-609,-Eden,-UT-84310","3612 N Huntsman Path, Unit 609, Eden, UT 84310",3612 N Huntsman Path,Unit 609,Eden,UT,84310,Weber,41.324364,-111.823717,...,2.0,1100.0,3485.0,2002.0,Inactive,2024-01-24T00:00:00.000Z,2024-04-06T00:00:00.000Z,2021-08-19T11:30:57.821Z,2024-04-05T00:00:00.000Z,73.0
17,"4608-W-Hemsworth-Ln-S,---2067,-Riverton,-UT-84096","4608 W Hemsworth Ln S, # 2067, Riverton, UT 84096",4608 W Hemsworth Ln S,# 2067,Riverton,UT,84096,Salt Lake,40.504021,-112.000793,...,2.5,2280.0,1307.0,2024.0,Inactive,2024-01-29T00:00:00.000Z,2024-04-06T00:00:00.000Z,2024-02-29T00:00:00.000Z,2024-04-05T00:00:00.000Z,68.0
24,"3155-S-Hidden-Valley-Dr,-Unit-208,-Saint-Georg...","3155 S Hidden Valley Dr, Unit 208, Saint Georg...",3155 S Hidden Valley Dr,Unit 208,Saint George,UT,84790,Washington,37.053494,-113.569428,...,2.5,1430.0,871.0,2006.0,Inactive,2024-02-10T00:00:00.000Z,2024-04-06T00:00:00.000Z,2024-02-10T00:00:00.000Z,2024-04-05T00:00:00.000Z,56.0


In [331]:
# Make a y variable with the price column
y = df['price']

In [332]:
# # Use get_dummies to make categorical variables into numerical.
# X = pd.get_dummies(X_full)

In [333]:
select_features = ['city', 'state', 'zipCode', 'county', 'latitude', 'longitude', 'propertyType', 'bedrooms', 'bathrooms', 'squareFootage', 'lotSize', 'yearBuilt',
'status', 'listedDate', 'removedDate', 'createdDate', 'daysOnMarket']

# select_features = ['formattedAddress','addressLine1']


In [334]:
select_features


['city',
 'state',
 'zipCode',
 'county',
 'latitude',
 'longitude',
 'propertyType',
 'bedrooms',
 'bathrooms',
 'squareFootage',
 'lotSize',
 'yearBuilt',
 'status',
 'listedDate',
 'removedDate',
 'createdDate',
 'daysOnMarket']

In [335]:
# Create another variable X_sel with only the columns in the "select_features" list.
# Ensure the columns in select_features exist in the dataframe
# select_features = [col for col in select_features if col in df.columns]
X = df[select_features]
X

,city,state,zipCode,county,latitude,longitude,propertyType,bedrooms,bathrooms,squareFootage,lotSize,yearBuilt,status,listedDate,removedDate,createdDate,daysOnMarket
8,Riverton,UT,84096,Salt Lake,40.503300,-112.001572,Townhouse,5.0,3.0,3622.0,1307.0,2024.0,Inactive,2024-01-16T00:00:00.000Z,2024-04-06T00:00:00.000Z,2024-02-14T00:00:00.000Z,81.0
11,Eagle Mountain,UT,84005,Utah,40.322655,-112.023788,Single Family,3.0,2.5,4054.0,9148.0,2024.0,Inactive,2024-01-18T00:00:00.000Z,2024-04-06T00:00:00.000Z,2024-01-19T00:00:00.000Z,79.0
13,Eden,UT,84310,Weber,41.324364,-111.823717,Condo,2.0,2.0,1100.0,3485.0,2002.0,Inactive,2024-01-24T00:00:00.000Z,2024-04-06T00:00:00.000Z,2021-08-19T11:30:57.821Z,73.0
17,Riverton,UT,84096,Salt Lake,40.504021,-112.000793,Townhouse,3.0,2.5,2280.0,1307.0,2024.0,Inactive,2024-01-29T00:00:00.000Z,2024-04-06T00:00:00.000Z,2024-02-29T00:00:00.000Z,68.0
24,Saint George,UT,84790,Washington,37.053494,-113.569428,Townhouse,3.0,2.5,1430.0,871.0,2006.0,Inactive,2024-02-10T00:00:00.000Z,2024-04-06T00:00:00.000Z,2024-02-10T00:00:00.000Z,56.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82505,Saint George,UT,84790,Washington County,37.049554,-113.554598,Townhouse,3.0,3.0,1465.0,871.0,2015.0,Inactive,2021-03-16T05:32:22.182Z,2022-08-12T17:58:47.827Z,2021-03-16T05:32:22.182Z,515.0
82508,Logan,UT,84341,Cache County,41.790873,-111.787867,Single Family,8.0,5.0,7036.0,26136.0,2005.0,Inactive,2022-04-09T09:22:56.444Z,2022-08-12T17:58:47.827Z,2022-04-09T09:22:56.444Z,126.0
82515,Duck Creek,UT,84762,Kane County,37.490615,-112.778916,Single Family,1.0,2.0,300.0,115870.0,2014.0,Inactive,2021-12-01T08:00:24.086Z,2022-08-12T17:58:47.827Z,2021-12-01T08:00:24.086Z,255.0
82522,Willard,UT,84340,Box Elder County,41.394288,-112.035649,Single Family,4.0,3.0,3326.0,11761.0,2022.0,Inactive,2022-06-29T09:43:18.329Z,2022-08-12T17:58:47.827Z,2022-06-29T09:43:18.329Z,45.0


In [336]:
# Set the target variable y.
y = df['price']
y


8         588900
11        632525
13        505000
17        501900
24        309900
          ...   
82505     335000
82508    1165000
82515     110000
82522     648000
82535     255000
Name: price, Length: 15115, dtype: object

In [337]:
# Now split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(11336, 17) (3779, 17) (11336,) (3779,)


## Train the models

In [338]:
print(X_train)


                   city state  zipCode             county   latitude  \
73523              Lehi    UT    84043        Utah County  40.373437   
20484         Bountiful    UT    84010              Davis  40.881230   
36654      Saint George    UT    84770         Washington  37.108984   
69645         Hurricane    UT    84737  Washington County  37.163891   
64785        West Point    UT    84015       Davis County  41.115162   
...                 ...   ...      ...                ...        ...   
25975        Heber City    UT    84032            Wasatch  40.492405   
71507       West Jordan    UT    84081   Salt Lake County  40.606659   
27070        Clearfield    UT    84015              Davis  41.100114   
3971   Saratoga Springs    UT    84045               Utah  40.312614   
37733            Murray    UT    84107          Salt Lake  40.639602   

        longitude   propertyType bedrooms bathrooms squareFootage  \
73523 -111.937115      Townhouse      3.0       2.5        2187.0 

In [339]:
print(y_train)

73523                      455990
20484    2023-10-31T00:00:00.000Z
36654                      519900
69645                      319900
64785                      574990
                   ...           
25975                      499900
71507    2022-10-02T10:32:11.262Z
27070    2023-09-09T00:00:00.000Z
3971                       963900
37733                      405000
Name: price, Length: 11336, dtype: object


In [340]:
# Print shapes for debugging
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (11336, 17)
y_train shape: (11336,)


In [342]:
# Create the models
model1 = LinearRegression()
model2 = xgb.XGBRegressor()

# Fit the first model to the full training data
model1.fit(X_train, y_train)

# Fit the second model to the select training data
model2.fit(X_train, y_train)


ValueError: could not convert string to float: 'Lehi'

## Evaluate the model

In [343]:
# Calculate the mean_squared_error and the r-squared value
# for the testing data for model 1 and model 2.
mse1 = mean_squared_error(y_test, model1.predict(X_test))
r21 = r2_score(y_test, model1.predict(X_test))
mse2 = mean_squared_error(y_test, model2.predict(X_test))
r22 = r2_score(y_test, model2.predict(X_test))


# Use our models to make predictions
y_pred1 = model1.predict(X_test)
y_pred2 = model2.predict(X_test)


# Score the predictions with mse and r2
scoremodel1 = model1.score(X_test, y_test) 
scoremodel2 = model2.score(X_test, y_test)


# Print the scores
print(f"Linear Regression Model Score: {scoremodel1}")
print(f"XGBRegressor Model Score: {scoremodel2}")
print(f"All Features:")
print(f"mean squared error (MSE): {mse1}")
print(f"R-squared (R2): {r21}")
print("---------------------")
print(f"Select Features:")
print(f"mean squared error (MSE): {mse2}")
print(f"R-squared (R2): {r22}")

ValueError: could not convert string to float: 'Ivins'

In [344]:
# Ask Firas if I need to do this.

# Provided code to create the adjusted r-squared function
def r2_adj(x, y, model):
    r2 = model.score(x,y)
    n_cols = x.shape[1]
    return 1 - (1 - r2) * (len(y) - 1) / (len(y) - n_cols - 1)

In [ ]:
# Calculate the adjusted r-squared value of the model.
adj_r2_model1 = r2_adj(X_test, y_test, model1)
adj_r2_model2 = r2_adj(X_test, y_test, model2)


ValueError: could not convert string to float: 'Ivins'

In [347]:
# Examine linear regression on the better training data using cross validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import pandas as pd

# Encode categorical variables
X_train_encoded = pd.get_dummies(X_train)
X_test_encoded = pd.get_dummies(X_test)

# Align the train and test dataframes by the columns
X_train_encoded, X_test_encoded = X_train_encoded.align(X_test_encoded, join='left', axis=1)

# Create a KFold object
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation on the linear regression model
cross_val_scores = cross_val_score(model1, X_train_encoded, y_train, scoring='r2', cv=kf)

# Print the cross-validation scores
print("Cross Validation Scores:", cross_val_scores)

# Calculate the mean of the cross-validation scores
cross_val_mean = cross_val_scores.mean()

# Print the mean of the cross-validation scores
print("Mean Cross Validation Score:", cross_val_mean)

# Examine XGBRegressor on the better training data using cross validation
# Perform cross-validation on the XGBRegressor model
cross_val_scores2 = cross_val_score(model2, X_train_encoded, y_train, scoring='r2', cv=kf)

# Print the cross-validation scores
print("Cross Validation Scores:", cross_val_scores2)

# Calculate the mean of the cross-validation scores
cross_val_mean2 = cross_val_scores2.mean()

# Print the mean of the cross-validation scores
print("Mean Cross Validation Score:", cross_val_mean2)

# Print the adjusted r-squared value of the model.
print(f"Adjusted R-squared value for Linear Regression Model: {adj_r2_model1}")
print(f"Adjusted R-squared value for XGBRegressor Model: {adj_r2_model2}")

# Save the model to disk
import joblib
joblib.dump(model1, 'model1.joblib')
joblib.dump(model2, 'model2.joblib')
joblib.dump(X_train_encoded, 'X_train.joblib')
joblib.dump(y_train, 'y_train.joblib')
joblib.dump(X_test_encoded, 'X_test.joblib')
joblib.dump(y_test, 'y_test.joblib')
joblib.dump(cross_val_mean, 'cross_val_mean.joblib')
joblib.dump(cross_val_mean2, 'cross_val_mean2.joblib')
joblib.dump(adj_r2_model1, 'adj_r2_model1.joblib')
joblib.dump(adj_r2_model2, 'adj_r2_model2.joblib')

# Load the model from disk
loaded_model1 = joblib.load('model1.joblib')
loaded_model2 = joblib.load('model2.joblib')
loaded_X_train = joblib.load('X_train.joblib')
loaded_y_train = joblib.load('y_train.joblib')
loaded_X_test = joblib.load('X_test.joblib')
loaded_y_test = joblib.load('y_test.joblib')
loaded_cross_val_mean = joblib.load('cross_val_mean.joblib')
loaded_cross_val_mean2 = joblib.load('cross_val_mean2.joblib')
loaded_adj_r2_model1 = joblib.load('adj_r2_model1.joblib')
loaded_adj_r2_model2 = joblib.load('adj_r2_model2.joblib')

# Print the loaded model
print(loaded_model1)
print(loaded_model2)
print(loaded_X_train)
print(loaded_y_train)
print(loaded_X_test)
print(loaded_y_test)
print(loaded_cross_val_mean)
print(loaded_cross_val_mean2)
print(loaded_adj_r2_model1)
print(loaded_adj_r2_model2)

# Print the loaded model score
print(loaded_model1.score(loaded_X_test, loaded_y_test))
print(loaded_model2.score(loaded_X_test, loaded_y_test))
print(loaded_cross_val_mean)
print(loaded_cross_val_mean2)
print(loaded_adj_r2_model1)
print(loaded_adj_r2_model2)

# Print the loaded model predictions
print(loaded_model1.predict(loaded_X_test))
print(loaded_model2.predict(loaded_X_test))


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\kerim\anaconda3\envs\dev\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\kerim\anaconda3\envs\dev\lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\kerim\anaconda3\envs\dev\lib\site-packages\sklearn\linear_model\_base.py", line 578, in fit
    X, y = self._validate_data(
  File "c:\Users\kerim\anaconda3\envs\dev\lib\site-packages\sklearn\base.py", line 650, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "c:\Users\kerim\anacond